# Define Data

In [ ]:
# imports 
import numpy as np
import pandas as pd 
import os,random
import warnings
warnings.filterwarnings("ignore")

from sklearn.preprocessing import StandardScaler, MinMaxScaler,MaxAbsScaler,RobustScaler

from sklearn import datasets
from sklearn.model_selection import train_test_split
from sklearn.model_selection import RandomizedSearchCV
from scipy.stats import uniform as sp_randFloat
from scipy.stats import randint as sp_randInt

from catboost import CatBoostClassifier

# variables
TRAIN_PATH = "../input/spaceship-titanic/train.csv"
TEST_PATH = "../input/spaceship-titanic/test.csv"
SAMPLE_SUBMISSION_PATH = "../input/spaceship-titanic/sample_submission.csv"
SUBMISSION_PATH = "submission.csv"

ID = "PassengerId"
TARGET = "Transported"

BOOL_COL = ["CryoSleep","VIP"]

SEED = 2022
def seed_everything(seed=SEED):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    
seed_everything()

# Preprocess Data

In [ ]:
def autoPreProcess(train,test,DROP_COLS,TARGET):
    train_len = len(train)

    train_test = pd.concat(objs=[train, test], axis=0).reset_index(drop=True)
    train_test = train_test.drop(DROP_COLS, axis = 1)

    def checkNull_fillData(df):
        for col in df.columns:
            if len(df.loc[df[col].isnull() == True]) != 0:
                if df[col].dtype == "float64" or df[col].dtype == "int64":
                    df.loc[df[col].isnull() == True,col] = df[col].median()
                elif col in BOOL_COL:
                    df.loc[df[col].isnull() == True,col] = df[col].mode()[0]
                else:
                    df.loc[df[col].isnull() == True,col] = "Missing"

    checkNull_fillData(train_test)
    
    for col in BOOL_COL:
        train_test[col] =  train_test[col].astype(int)

    str_list = [] 
    num_list = []
    for colname, colvalue in train_test.iteritems():
        if colname == TARGET:
            continue
            
        if type(colvalue[1]) == str:
            str_list.append(colname)
        else:
            num_list.append(colname)

    train_test = pd.get_dummies(train_test, columns=str_list)

    scaler = MinMaxScaler(feature_range=(-3,3))
    train_test[num_list] = scaler.fit_transform(train_test[num_list])
    
    train = train_test[:train_len]
    test = train_test[train_len:]

    test.drop(labels=[TARGET],axis = 1,inplace=True)
    
    return train,test

train = pd.read_csv(TRAIN_PATH)
train[TARGET] = train[TARGET].astype(int)

test = pd.read_csv(TEST_PATH)
DROP_COLS = ['PassengerId', 'Name', 'Cabin']

train,test = autoPreProcess(train,test,DROP_COLS,TARGET)
train.head()

# Build Model

In [ ]:
RS_CV = 3
RS_N_ITER = 10
RS_N_JOBS = -1
RS_SCORING = 'accuracy'

X = train.drop([TARGET],axis=1)
y = train[TARGET]

model = CatBoostClassifier()
parameters = {'depth'         : sp_randInt(1, 10),
              'learning_rate' : sp_randFloat(),
              'iterations'    : sp_randInt(10, 1000)
             }

rs = RandomizedSearchCV(estimator=model, 
                           param_distributions = parameters,
                           scoring=RS_SCORING,
                           cv = RS_CV, 
                           n_iter = RS_N_ITER, 
                           n_jobs=RS_N_JOBS)
rs.fit(X, y)

# best model 
print(rs.best_params_)
bestModel = rs.best_estimator_

# Predict Data

In [ ]:
#predict
X_test = test
pred_test = bestModel.predict(X_test)
#submit
sub = pd.read_csv(SAMPLE_SUBMISSION_PATH)
sub[TARGET] = pred_test.astype(int)
sub[TARGET] = pred_test.astype(bool)
sub.to_csv(SUBMISSION_PATH, index=False)
sub.head()